<a href="https://colab.research.google.com/github/dlinero/AI-canopy-height-estimation/blob/main/Downloading_and_structuring_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q earthengine-api geemap h5py tifffile numpy tqdm rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 54.8 MB/s eta 0:00:00


In [ ]:
# Give access to Google Earth Engine
import ee
ee.Authenticate()
ee.Initialize(project='ee-linerotriana')
print("✔ EE initialized")


✔ EE initialized


In [ ]:
import os
from tqdm import tqdm
import numpy as np

# Give access to google drive
from google.colab import drive
drive.mount('/content/drive')

# Define number of tiles and their dimensions
NUM_TILES=1000
TILE_HALF_SIZE_M = 320 # 64×64 px


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create a function to randomly sampled tiles across the TSMBF biome
def get_random_tile_geometries(num_tiles=NUM_TILES, seed=42):
    eco = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
    biome_fc = eco.filter(
        ee.Filter.eq("BIOME_NAME", "Tropical & Subtropical Moist Broadleaf Forests")
    )
    region = biome_fc.geometry()

    pts = ee.FeatureCollection.randomPoints(region, points=num_tiles, seed=seed)

    def to_tile(f):
        g = f.geometry()
        return ee.Feature(g.buffer(TILE_HALF_SIZE_M).bounds())

    return pts.map(to_tile)

tiles_fc = get_random_tile_geometries()
tile_list = tiles_fc.toList(NUM_TILES)

print("✔ Tile geometries ready")


✔ Tile geometries ready


In [ ]:
# Create a function to identify the first available Sentinel 1 image for 2020 per tile and
# only extract VV and VH channels
def find_first_vvvh_2020(tile_geom):
    """Return the first VV/VH S1 image in 2020 over the tile."""

    col = (
        ee.ImageCollection("COPERNICUS/S1_GRD")
        .filterBounds(tile_geom)
        .filterDate("2020-01-01", "2021-01-01")
        .filter(ee.Filter.eq("instrumentMode", "IW"))
        .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
        .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH"))
        .sort("system:time_start")
        .limit(1)
    )

    img = col.first()       # lazy; safe
    return img.select(["VV", "VH"])
line

In [ ]:
# Function to stack sentinel 1 VV and VH, with elevation, slope and canopy height
def build_tile_image(tile_geom):

    # Sentinel-1: already in dB
    s1img = find_first_vvvh_2020(tile_geom)

    s1_db = (
        s1img
        .clip(tile_geom)
        .rename(["S1_VV_dB", "S1_VH_dB"])
        .toInt16()
    )

    # NASADEM elevation + slope
    elev = (
        ee.Image("NASA/NASADEM_HGT/001")
        .select("elevation")
        .clip(tile_geom)
        .rename("Elevation")
        .toInt16()
    )

    slope = ee.Terrain.slope(elev).rename("Slope").toInt16()

    # ETH canopy height (m -> cm)
    canopy = (
        ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1")
        .clip(tile_geom)
        .multiply(100)
        .rename("CH_2020_cm")
        .toInt16()
    )

    return (
        s1_db
        .addBands(elev)
        .addBands(slope)
        .addBands(canopy)
    )


In [ ]:
# Run functions for each randomly sampled tile
tasks = []

for i in range(NUM_TILES):

    tile_geom = ee.Feature(tile_list.get(i)).geometry()
    tile_img = build_tile_image(tile_geom)

    if tile_img is None:
        print(f"No VV/VH found for tile {i}. Picking replacement…")
        replacement = get_random_tile_geometries(1, seed=100+i)
        tile_geom = ee.Feature(replacement.first()).geometry()
        tile_img = build_tile_image(tile_geom)

    task = ee.batch.Export.image.toDrive(
        image = tile_img,
        description = f"tile_{i}",
        folder = "tropical_tiles_raw",
        fileNamePrefix = f"tile_{i}",
        region = tile_geom,
        dimensions = [64, 64],
        fileFormat = "GeoTIFF",
        maxPixels = 1e12
    )

    task.start()
    print(f"✔ Started export task for tile {i}")

print("\n📌 Monitor tasks:")
print("https://code.earthengine.google.com/tasks")


✔ Started export task for tile 0
✔ Started export task for tile 1
✔ Started export task for tile 2
✔ Started export task for tile 3
✔ Started export task for tile 4
✔ Started export task for tile 5
✔ Started export task for tile 6
✔ Started export task for tile 7
✔ Started export task for tile 8
✔ Started export task for tile 9
✔ Started export task for tile 10
✔ Started export task for tile 11
✔ Started export task for tile 12
✔ Started export task for tile 13
✔ Started export task for tile 14
✔ Started export task for tile 15
✔ Started export task for tile 16
✔ Started export task for tile 17
✔ Started export task for tile 18
✔ Started export task for tile 19
✔ Started export task for tile 20
✔ Started export task for tile 21
✔ Started export task for tile 22
✔ Started export task for tile 23
✔ Started export task for tile 24
✔ Started export task for tile 25
✔ Started export task for tile 26
✔ Started export task for tile 27
✔ Started export task for tile 28
✔ Started export task fo